## 1: Building a stack

Let GPT come up with steps to problem solving

We'll start off with the loading of our environmental variables etc.

In [1]:
import os, openai, json
openai.api_key = os.environ['OPENAI_KEY']
try:
    openai.organization = os.environ['OPENAI_ORG']
    print('Organization Added')
except:
    print('No Organization')
    
#Run this if you want to use your personal instead of default org
openai.organization=None

os.makedirs('prompts', exist_ok=True)

Organization Added


Our query wrapper

In [2]:
def query(prompt, full=False, **kwargs):
    modelKwargs = {
        'temperature':0,
        'engine':'davinci',
        'max_tokens':10,
        'stop':['\n']
    }
    
    for kwarg in kwargs:
        modelKwargs[kwarg] = kwargs[kwarg]
    
    completion = openai.Completion.create(prompt=prompt, **modelKwargs)
    
    with open('prompts/{}.json'.format(completion['id']), 'w') as fh:
        json.dump(completion, fh, indent=4)
    
    if full:
        return completion
    if 'n' in modelKwargs:
        return [x['text'] for x in completion['choices']]
    return completion['choices'][0]['text']

In [3]:
query('1+1=')

'2.'

So when we're solving a task, we've seen already that we can use intermediary steps to help GPT generate the final answer, e.g., with math in the introductory notebook 2, we first calculate the digits that we'll need to write out.

In [5]:
prompt = """Task: Spell out the sum of each math problem.

Question: 40 + 50
Answer: 90
Spelled Out: Ninety

Question: 1+1
Answer: 2
Spelled Out: Two

Question: 50 + 80
Answer: 130
Spelled Out: One hundred and twenty

Question: 12 + 52
Answer: 64
Spelled Out: Sixty Four

Question: {}
Answer:"""
query(prompt.format('450+650'), stop=['\n\n'])

' 1100\nSpelled Out: One Thousand and Ten'

(oddly, it got that wrong but that's not the point)

Sometimes, rather than have GPT wander on its own towards a problem, we might want to let GPT come up with the intermediary steps on its own rather than follow a set that we hard code for it. There's a lot of reason there might be intermediary steps, such as extracting values from a database or any sort of complex exercise.

Actually, the example above is a good example of this. It got 1100 as `One Thousand and Ten` and we might want to do that in a seperate step. So let's split the prompt up into two different calls. We can start by hard coding it.

In [46]:
prompt1 = """Sum: 50 + 25
Result: 75

Sum: 520 + 310
Result: 830

Sum: 1,540 + 4,200
Result: 5,740

Sum: {}
Result:"""

In [18]:
prompt2 = """Digits: 540
Spelled: Five hundred forty.

Digits: 5,203
Spelled: Five thousand, two hundred and three.

Digits: 9,100
Spelled: Nine thousand, one hundred.

Digits: {}
Spelled:"""

Then we can just create a pipeline where we feed the result of the prior answer to the next prompt, e.g

In [11]:
def smartAdd(int1, int2):
    firstResult = query(prompt1.format('{} + {}').format(int1, int2)).strip()
    print('First Result: ', firstResult)
    secondResult = query(prompt2.format(firstResult))
    print('Second Result: ', secondResult)

In [12]:
smartAdd(52, 43)

First Result:  : 95
Second Result:  : Ninety-five


In [13]:
smartAdd('1,421', '300')

First Result:  : 1,721
Second Result:  : One thousand, seven hundred and twenty-one


In [19]:
smartAdd(450, 650)

First Result:  : 1,100
Second Result:   One thousand, one hundred.


In [20]:
smartAdd(4500, 6500)

First Result:  : 11,000
Second Result:   Eleven thousand.


In [23]:
smartAdd(4501, 6540)

First Result:  : 11141
Second Result:   One hundred and eleven thousand four hundred and one.


It turns out this still has some bugs in it, but including commas can help it keep track of where the places are (although apparently we need to give it examples for how we want things spelled)

In [22]:
smartAdd('4,501', '6,540')

First Result:  : 11,041
Second Result:   Eleven thousand, forty-one.


Cool, so we can chain multiple prompts together, but we have to come up with the order of steps on our own. To do this, we can put them into a stack. A stack is basically an array, where we'll stick a bunch of things into a queue of things to do, and then we'll remove them until we finish our task.

In [48]:
stack = []
stack.append(prompt1)
stack.append(prompt2)

In [49]:
result = '4,501 + 6,540'

while len(stack) > 0:
    nextInstruction = stack.pop(0)
    result = query(nextInstruction.format(result)).strip()
    print(result)

11,041
Eleven thousand, forty-one.


So now we just need a prompt to make the stack! Well, in this case, we had to format it I guess, but let's go for a general problem solving prompt

In [72]:
problemPrompt = """Each question may require multiple steps

Question: What is 5+5
Steps:
1: Answer: Solve 5+5

Question: What is the zip code of my house
Steps:
1: Need: Input address (street, city, state/country)
2: Answer: Solve for zip code of above.

Question: What is the capital of France
Steps:
1: Answer: Solve capital of France

Question: Who wrote the book I am holding
Steps:
1: Input: Input the title of the book are you holding
2: Answer: Solve for author of above

Question: {}
Steps:"""

In [64]:
query(problemPrompt.format('What is 1+1'), stop=['\n\n'])

'\n1: Answer: Solve 1+1'

In [66]:
query(problemPrompt.format('What is this made of'), stop=['\n\n'], max_tokens=30)

'\n1: Input: What is this\n2: Answer: Solve for material of above'

This seems to work but there might be better ways to do it. Now, let's take each part and stick it in the stack.

In [73]:
r = query(problemPrompt.format('What is the zip code of my friend\'s house'), stop=['\n\n'], max_tokens=30)

In [74]:
print(r)


1: Input: Input the address of your friend
2: Answer: Solve for zip code of above.


In [83]:
stack = []
for line in r.split('\n'):
    if line.find(":") > -1:
        stack.append(line[line.index(":")+1:].strip())

In [84]:
print(stack)

['Input: Input the address of your friend', 'Answer: Solve for zip code of above.']


Great, now we've got a stack, but need to traverse it now. So we can create two functions. The first, we want to be able to take an input. We can do that in python by just asking for an input as such:

In [86]:
newVar = input('Put something in: ', )
print(newVar)

Put something in: Foo
Foo


Now when we traverse the stack, we can use input as the prompt

In [89]:
for line in stack:
    if line.startswith('Input'):
        print(input(line[line.find(":")+1:].strip() + ': ', ))

Input the address of your friend: 710 3rd St, San Francisco, CA
710 3rd St, San Francisco, CA


So for inputs, we can do that, and then we can just have a general question answering prompt for the ones that require answers

In [137]:
prompt = """Question: Who is the author of this book?
Input: What is the title of the book?
War and Peace
Solve for author of above
Answer: Leo Tolstoy

Question: {}
"""

In [138]:
def buildStack(gptResponse):
    stack = []
    for line in gptResponse.split('\n'):
        if line.find(":") > -1:
            stack.append(line[line.index(":")+1:].strip())
    return stack

In [143]:
myQuestion = input('Enter your question: ', )

Enter your question: What is my friends's house's zip code?


In [144]:
gptResponse = query(problemPrompt.format(myQuestion), stop=['\n\n'], max_tokens=30)

In [145]:
myStack = buildStack(gptResponse)

So now we can traverse the stack

In [147]:
myStack

["Input: Input the address of your friend's house",
 'Answer: Solve for zip code of above.']

In [148]:
finalPrompt = prompt.format(myQuestion)

while len(myStack) > 0:
    nextInstruction = myStack.pop(0)
    if (nextInstruction.startswith('Input:')):
        formattedInstruction = nextInstruction[nextInstruction.find(":")+1:].strip()
        finalPrompt += 'Input: ' + formattedInstruction + '\n'
        finalPrompt += input(formattedInstruction + ': ', )
    else:
        finalPrompt += nextInstruction[nextInstruction.find(":")+1:].strip()
        finalPrompt += '\nAnswer:'
        newResult = query(finalPrompt.strip())
        print(newResult)
        finalPrompt += newResult
        
    finalPrompt +='\n'

Input the address of your friend's house: 710 3rd St, San Francisco, CA
 94103


Of course, that Zip code's wrong but it gets the point across. Can you come up with complicated systems you might need to come up with steps to solve? It turns out coming up with steps on the fly is actually pretty hard!